In [38]:
import numpy as np
import torch.nn
from sklearn.model_selection import train_test_split
from scipy.io import loadmat

In [39]:
mnist = loadmat('mnist_49_3000.mat')
X = mnist['x']
y = mnist['y'][0]

X_train, X_test, y_train, y_test = train_test_split(X.T, y, test_size=0.25, random_state=42)

In [52]:
# change -1 to 0 before fitting
def converter(inp):
    if inp > 0:
        return 1
    else:
        return 0

y_train_mod = np.reshape(np.array([converter(val) for val in y_train]), (len(y_train), 1))
print(y_train_mod.shape)
y_test_mod = np.array([converter(val) for val in y_test])


(2250, 1)


In [53]:
class LogisticRegression:
    def __init__(self, lr=0.01, lamb=10, num_iter=1000, fit_intercept=True):
        self.lr = lr
        self.lamb = lamb
        self.num_iter = num_iter
        self.fit_intercept = fit_intercept
        
    def __add_intercept(self, X):
        intercept = np.ones((X.shape[0], 1))
        return np.concatenate((intercept, X), axis=1)
    
    def __sigmoid(self, z):
        '''Exercise - Define Sigmoid Function here. z is a vector'''
        return 1 / (1 + np.exp(-z))
    
    def fit(self, X, y):
        if self.fit_intercept:
            X = self.__add_intercept(X)
        
        # weights initialization - all zeros
        '''Exercise - Initialize theat. What should be the dimension of theta?'''
        self.theta = np.zeros((X.shape[1], 1))
        loss = torch.nn.BCELoss()

        for i in range(self.num_iter):
            '''Exercise - Implement x^T(theta)'''
            z = np.matmul(X, self.theta)
            h = self.__sigmoid(z)

            print(h.shape)
            print(y.shape)

            loss_step = loss(h, y)
            grad = loss_step.backward()
            
            '''Exercise - [Critical Step] - Using the formula in question prompt define gradient in vector form'''
            #gradient = -1 / ((1 + np.exp(-z)) ** 2) * (-np.exp(-z)) * X
            
            '''Exercise - [Critical Step] - Update theta using the gradient and learning rate'''
            self.theta -= grad * self.theta
    
    def predict_prob(self, X):
        if self.fit_intercept:
            X = self.__add_intercept(X)
        
        return self.__sigmoid(np.dot(X, self.theta))
    
    def predict(self, X, threshold):
        return self.predict_prob(X) >= threshold

In [54]:
# Initialize and train the model
'''Exercise - Select and appropriate lambda parameter in ...'''
model = LogisticRegression(lamb=0.01)

%time model.fit(X_train, y_train_mod) #%time prints out the execution time for the command

(2250, 1)
(2250, 1)


TypeError: 'int' object is not callable

In [ ]:
# Generate Predictions
preds = model.predict(X_test, 0.5)
(preds == y_test_mod).mean()